In [1]:
import geopandas as gpd
import os
import datetime
import pandas as pd

In [4]:
#set data path
data_path = os.path.join(os.getcwd(), "data","data")

#read usps
usps = pd.read_csv(data_path + r"\20250731_USPS_parcel_deliverability.csv")

#read demolitions
demolitions = pd.read_csv(data_path + r'\20250731_Completed_Residential_Demolitions.csv')

#read DWSD water/sewer
dwsd = pd.read_csv(data_path + r"\20250731_dwsd_account_2020-2024.csv")

#read ODM
odm = pd.read_csv(data_path + r"\20250731_Order to Demolish.csv")

vacant_structures = pd.read_csv(data_path + r"\20250731_vacant_structures_geocoded.csv")

In [5]:

parcels = gpd.read_file(os.getcwd() + r"\20250728_CODLayers.gdb\CODBaseUnitLayers.gdb")
dlba_survey = pd.read_excel(os.getcwd() + r"\20250527_DLBA_blight_survey_data_UM_Detroit\20250527_DLBA_survey_data_UM_Detroit.xlsx")

#read ago data
fire_incidents = gpd.read_file("https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/Fire_Incidents/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")
trade_permits = gpd.read_file("https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/bseed_trades_permits/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")
building_permits = gpd.read_file("https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/bseed_building_permits/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")
demolitions = gpd.read_file("https://services2.arcgis.com/qvkbeam7Wirps6zC/arcgis/rest/services/parcel_file_current/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson")

c:\Users\gregg.saldutti\AppData\Local\miniforge3\lib\site-packages\pyogrio\geopandas.py:275: UserWarning: More than one layer found in 'CODBaseUnitLayers.gdb': 'Parcels2025' (default), 'Buildings', 'Address_Points'. Specify layer parameter to avoid this warning.
  result = read_func(


Subset relevant fields from parcel data and keep only parcels in DLBA survey

In [6]:
#select relevant parcel fields
parcel_fields = ["parcel_number",'year_built', 'tax_status','assessed_value', 'is_improved','total_floor_area', 'property_class_desc']
attributed = parcels.loc[:,parcel_fields]

#remove non-residential classed properties (DLBA survey data only covers residential properties)
attributed = dlba_survey.merge(attributed, how='left', left_on='PARCEL_ID', right_on='parcel_number')

In [7]:
#keep relevant fields
dlba_fields = ['PARCEL_ID', 'DATE_SURVEYED','FIELD_DETERMINATION', 'SURVEYOR_NOTES', 'HAS_STRUCTURE', 'IS_OCCUPIED',
               'FACADE_SIDING_CONDITION', 'FIRE_DAMAGE_CONDITION', 'ROOF_CONDITION', 'OPENINGS_CONDITION',
               'IS_OPEN_TO_TRESPASS', 'PORCH_STEPS_CONDITION']

keep_fields = parcel_fields + dlba_fields
attributed = attributed[keep_fields]

#na field determinations indicate different methodology and should probably not be retained
attributed = attributed[~attributed.FIELD_DETERMINATION.isna()]

Use fire incidents for indvidual properties, as well as buffering properties to find fire incidents in 500ft radius.

We will have to create dummy variables for incidents at a property in last (3) years.

In [8]:
#convert datetime
fire_incidents['unit_cleared_at'] = pd.to_datetime(fire_incidents['unit_cleared_at'], unit='ms')
#join fire incidents to parcels
attributed = attributed.merge(fire_incidents[['parcel_id', 'unit_cleared_at', 'address_id']], 
                              how='left', 
                              left_on='parcel_number', 
                              right_on='parcel_id',
                              suffixes=("", "_fire"))

Join permits

In [9]:
#join trade permits
attributed = attributed.merge(trade_permits[['parcel_id', 'record_id','permit_type', 'issued_date', 'address_id']], 
                              how='left',
                              left_on='parcel_number',
                              right_on='parcel_id',
                              suffixes=("","_trd"))

In [10]:
#join buildling permits
attributed = attributed.merge(building_permits[['parcel_id', 'issued_date', 'record_id', 'address_id']],
                              how='left',
                              left_on='parcel_number',
                              right_on='parcel_id',
                              suffixes=("", "_bld"))

In [11]:
usps

,VACANT,PARCEL_ID,MAILER_EXPORT_DATE,count
0,N,CONDO BUILDING,2020-05-01,6743.0
1,N,CONDO BUILDING,2021-10-01,1049.0
2,Y,CONDO BUILDING,2020-05-01,906.0
3,N,CONDO BUILDING,2022-10-01,640.0
4,N,CONDO BUILDING,2023-11-01,640.0
...,...,...,...,...
1875537,NaN,20014415.,2020-05-01,1.0
1875538,N,20014834.,2020-05-01,1.0
1875539,NaN,20017766.,2020-05-01,1.0
1875540,Y,20018493.,2020-05-01,1.0


Join USPS

In [13]:
attributed = attributed.merge(usps[usps.PARCEL_ID != 'CONDO BUILDING'], how = 'left', left_on = 'parcel_number', right_on = 'PARCEL_ID')

Join DWSD

In [ ]:
dwsd
attributed = attributed.merge(dwsd, how='left', left_on = 'dwsd_parcel_number, right_on = 'parcel_number )

,dwsd_parcel_number,address_number,address_street_name,account_active_code,water_status
0,16036665,18610.0,KENTUCKY ST,ACTIVE,WATER ON
1,22017129-34,17256.0,W 7 MILE RD,NaN,WTR ON
2,8007096,6178.0,AVERY ST,ACTIVE,WATER OFF
3,16023085,19981.0,PRAIRIE ST,NaN,WTR ON
4,02001628.,119.0,BURLINGAME ST,ACTIVE,Water On
...,...,...,...,...,...
625663,19006894,5710.0,MCCLELLAN ST,ACTIVE,WATER ON
625664,22077373.,6467.0,ROSEMONT AVE,ACTIVE,Water On
625665,22038006-7,18080.0,STRATHMOOR ST,ACTIVE,WATER ON
625666,02004206.002L,18431.0,WARRINGTON DR,ACTIVE,Water On


Deduping
- remove all straight dupes
- where multiple permits exist, keep the lastest one
- where multiple fire incidents exist, keep latest one

In [119]:
attributed = attributed[~attributed.duplicated()]

In [123]:
attributed[~attributed.issued_date.isna()]

,parcel_number,year_built,tax_status,assessed_value,is_improved,total_floor_area,property_class_desc,PARCEL_ID,DATE_SURVEYED,FIELD_DETERMINATION,...,address_id,parcel_id_trd,record_id,permit_type,issued_date,address_id_trd,parcel_id_bld,issued_date_bld,record_id_bld,address_id_bld
603,12012121.,1941.0,TAXABLE,34600.0,1.0,2321.0,RESIDENTIAL-IMPROVED,12012121.,2023-10-05 00:00:00,No Action (Salvage),...,NaN,12012121.,MEC2019-06240,Mechanical Permit,2019-12-09,28191.0,NaN,NaT,NaN,NaN
5498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22087888.,2023-08-23 00:00:00,No Action (Salvage),...,NaN,None,MEC2019-06343,Mechanical Permit,2019-12-16,NaN,NaN,NaT,NaN,NaN
5499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22087888.,2023-08-23 00:00:00,No Action (Salvage),...,NaN,None,MEC2019-06422,Mechanical Permit,2019-12-16,NaN,NaN,NaT,NaN,NaN
5500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22087888.,2023-08-23 00:00:00,No Action (Salvage),...,NaN,None,ELE2019-05051,Electrical Permit,2019-12-16,256949.0,NaN,NaT,NaN,NaN
5501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22087888.,2023-08-23 00:00:00,No Action (Salvage),...,NaN,None,MEC2019-06552,Mechanical Permit,2019-12-17,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77086,22108657.,NaN,TAXABLE,900.0,0.0,NaN,RESIDENTIAL-VACANT,22108657.,2023-09-13 00:00:00,No Action (Salvage),...,NaN,22108657.,ELE2019-05036,Electrical Permit,2019-12-16,99063.0,NaN,NaT,NaN,NaN
80207,22060392.,1939.0,TAXABLE,25000.0,1.0,963.0,RESIDENTIAL-IMPROVED,22060392.,NaN,No Action (Salvage),...,NaN,22060392.,ELE2019-04677,Electrical Permit,2019-11-20,142467.0,NaN,NaT,NaN,NaN
80208,22060392.,1939.0,TAXABLE,25000.0,1.0,963.0,RESIDENTIAL-IMPROVED,22060392.,NaN,No Action (Salvage),...,NaN,22060392.,PLM2019-02231,Plumbing Permit,2019-11-21,142467.0,NaN,NaT,NaN,NaN
83182,16011007.,1915.0,TAXABLE,21100.0,1.0,2157.0,RESIDENTIAL-IMPROVED,16011007.,2024-11-04 00:00:00,Noticeable Evidence of Blight,...,NaN,16011007.,ELE2019-05018,Electrical Permit,2019-12-13,150634.0,NaN,NaT,NaN,NaN
